# TensorFlow Assignment: Reinforcement Learning (RL)

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: Khoi D. Vo </br>
Date: June 27, 2018

### Short answer

1\. One of the fundamental challenges of reinforcement learning is balancing *exploration* versus *exploitation*. What do these two terms mean, and why do they present a challenge?

For an RL agent, it often starts off naive in a brand new enviornment that it knows nothing about, other than the fact that the environment can provide it with feedback in the form of reward/punishment. It is often the case that exploration is necessary for an RL agent to source enough information about its available actions given a state and the potential state transitions that come about given an action. This allows the RL agent to learn a policy and optimize its value function. However, the purpose of an RL agent is to maximize its reward and at some point in its existence in the environment, it must exploit and take advantage of what it already knows. Exploitation could be good for the RL agent for it to capitalize on a given set of knowledge that it has gained from exploration. However, in non-static environments, exploration is necessary for the RL agent to not get stuck with a suboptimal policy. As an example, in the e-greedy implemention, it's always good to have a small random chance to influence the RL agent to explore, which could allow it to update its values accordingly in a non-static environment.

More concretely put - an animal can forage around in the forest for food. It lands in a patch that is rich with food and exploits this area. However, given the non-static nature of its environment (e.g., seasons, migration), the animal staying at the same patch is not optimal and it must explore to find different patches of food as the environment changes. It seems like biologically speaking, all biological agents have that innate explore/exploit tradeoff as it's crucial to survival. It's quite cool to see this aspect of life be integrated into an RL agent, as well.

2\. Another fundamental reinforcement learning challenge is what is known as the *credit assignment problem*, especially when rewards are sparse. 
What do we mean by the phrase, and why does this make learning especially difficult?
How does this interact with reward function design, where we have to be careful that our reward captures the true objective?

It's not always the case that every action given a state leads to an actionable outcome, aka. feedback from the environment. RL updating is all about back-propagation and in the case of the lake example we had today, a given RL agent will only receive feedback once it goes from the start to finish. Here, the first time the RL agent finishes, the penultimate state-action pair are updated and nothing more. This credit is then transferred back and back as the RL agent revisits the state in the future. Some of this credit assigment problem can be aleviated if we consider model-based RL, rather than model-free RL that we went over today.

### Deep SARSA Cart Pole

[SARSA (state-action-reward-state-action)](https://en.wikipedia.org/wiki/State–action–reward–state–action) is another Q value algorithm that resembles Q-learning quite closely:

Q-learning update rule:
\begin{equation}
Q_\pi (s_t, a_t) \leftarrow (1 - \alpha) \cdot Q_\pi(s_t, a_t) + \alpha \cdot \big(r_t + \gamma \max_a Q_\pi(s_{t+1}, a)\big)
\end{equation}

SARSA update rule:
\begin{equation}
Q_\pi (s_t, a_t) \leftarrow (1 - \alpha) \cdot Q_\pi(s_t, a_t) + \alpha \cdot \big(r_t + \gamma Q_\pi(s_{t+1}, a_{t+1})\big)
\end{equation}

Unlike Q-learning, which is considered an *off-policy* network, SARSA is an *on-policy* algorithm. 
When Q-learning calculates the estimated future reward, it must "guess" the future, starting with the next action the agent will take. In Q-learning, we assume the agent will take the best possible action: $\max_a Q_\pi(s_{t+1}, a)$. SARSA, on the other hand, uses the action that was actually taken next in the episode we are learning from: $Q_\pi(s_{t+1}, a_{t+1})$. In other words, SARSA learns from the next action he actually took (on policy), as opposed to what the max possible Q value for the next state was (off policy).

Build an RL agent that uses SARSA to solve the Cart Pole problem. 

*Hint: You can and should reuse the Q-Learning agent we went over earlier. In fact, if you know what you're doing, it's possible to finish this assignment in about 30 seconds.*

In [1]:
# Based on: https://gym.openai.com/evaluations/eval_EIcM1ZBnQW2LBaFN6FY65g/

import random
import gym
import math
import numpy as np
import tensorflow as tf
from collections import deque

class DQNCartPoleSolver():
    def __init__(self, n_episodes=1000, n_win_ticks=195, max_env_steps=None, gamma=1.0, epsilon=1.0, epsilon_min=0.01, epsilon_log_decay=0.995, alpha=0.01, alpha_decay=0.01, batch_size=64, monitor=False, quiet=False):
        self.memory = deque(maxlen=100000)
        self.env = gym.make('CartPole-v0')
        if monitor: self.env = gym.wrappers.Monitor(self.env, '../data/cartpole-1', force=True)
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_log_decay
        self.alpha = alpha
        self.alpha_decay = alpha_decay
        self.n_episodes = n_episodes
        self.n_win_ticks = n_win_ticks
        self.batch_size = batch_size
        self.quiet = quiet
        if max_env_steps is not None: self.env._max_episode_steps = max_env_steps

        # Init model
        self.state_ = tf.placeholder(tf.float32, shape=[None, 4])
        h = tf.layers.dense(self.state_, units=24, activation=tf.nn.tanh)
        h = tf.layers.dense(h, units=48, activation=tf.nn.tanh)
        self.Q = tf.layers.dense(h, units=2)
        
        self.Q_ = tf.placeholder(tf.float32, shape=[None, 2])
        loss = tf.losses.mean_squared_error(self.Q_, self.Q)
        self.global_step = tf.Variable(0, name='global_step', trainable=False)
        lr = tf.train.exponential_decay(0.01, self.global_step, 0.995, 1)
        self.train_step = tf.train.AdamOptimizer(lr).minimize(loss, global_step=self.global_step)
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

    def remember(self, state, action, reward, next_state, action_next_state, done):
        self.memory.append((state, action, reward, next_state, action_next_state, done)) # added action_next_state (KDV)

    def choose_action(self, state, epsilon):
        return self.env.action_space.sample() if (np.random.random() <= epsilon) else np.argmax(self.sess.run(self.Q, feed_dict={self.state_: state}))

    def get_epsilon(self, t):
        return max(self.epsilon_min, min(self.epsilon, 1.0 - math.log10((t + 1) * self.epsilon_decay)))

    def preprocess_state(self, state):
        return np.reshape(state, [1, 4])

    def replay(self, batch_size):
        x_batch, y_batch = [], []
        minibatch = random.sample(
            self.memory, min(len(self.memory), batch_size))
        for state, action, reward, next_state, action_next_state, done in minibatch: 
            y_target = self.sess.run(self.Q, feed_dict={self.state_: state})
            y_target_next = self.sess.run(self.Q, feed_dict={self.state_: next_state}) # get the possible Q values for the next state

            y_target[0][action] = reward if done else reward + self.gamma * y_target_next[0][action_next_state] # based on SARSA, update with the next state action once in new state
            x_batch.append(state[0])
            y_batch.append(y_target[0])
        
        self.sess.run(self.train_step, feed_dict={self.state_: np.array(x_batch), self.Q_: np.array(y_batch)})

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def run(self):
        scores = deque(maxlen=100)

        for e in range(self.n_episodes):
            state = self.preprocess_state(self.env.reset())
            done = False
            i = 0
            while not done:
                if e % 100 == 0 and not self.quiet:
                    self.env.render()
                action = self.choose_action(state, self.get_epsilon(e))
                next_state, reward, done, _ = self.env.step(action)
                
                next_state = self.preprocess_state(next_state)
                action_next_state = self.choose_action(next_state, self.get_epsilon(e)) # added action_next_state (KDV)
                    
                self.remember(state, action, reward, next_state, action_next_state, done)
                state = next_state
                
                i += 1

            scores.append(i)
            mean_score = np.mean(scores)
            if mean_score >= self.n_win_ticks and e >= 100:
                if not self.quiet: print('Ran {} episodes. Solved after {} trials ✔'.format(e, e - 100))
                return e - 100
            if e % 100 == 0 and not self.quiet:
                print('[Episode {}] - Mean survival time over last 100 episodes was {} ticks.'.format(e, mean_score))

            self.replay(self.batch_size)
        
        if not self.quiet: print('Did not solve after {} episodes 😞'.format(e))
        return e

if __name__ == '__main__':
    agent = DQNCartPoleSolver()
    agent.run()

/Users/KDV/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
[Episode 0] - Mean survival time over last 100 episodes was 19.0 ticks.
[Episode 100] - Mean survival time over last 100 episodes was 29.43 ticks.
[Episode 200] - Mean survival time over last 100 episodes was 95.39 ticks.
[Episode 300] - Mean survival time over last 100 episodes was 164.66 ticks.
[Episode 400] - Mean survival time over last 100 episodes was 134.99 ticks.
[Episode 500] - Mean survival time over last 100 episodes was 141.07 ticks.
[Episode 600] - Mean survival time over last 100 episodes was 177.54 ticks.
Ran 658 episodes. Solved after 558 trials ✔


Note: you should be able to find that SARSA works much better for the demo we went over during lecture.
This is not necessarily a general result.
Q-learning and SARSA tend to do better on different kinds of problems.